SAMRefiner
=====

**SAMRefiner: Taming Segment Anything Model for Universal Mask Refinement**

 * Paper: https://arxiv.org/abs/2502.06756

![SAMRefiner Overview](../assets/samrefiner_overview.jpg)

 * Installation

```bash
git clone https://github.com/linyq2117/SAMRefiner.git SAMRefiner_repo

# create conda env
conda create -n SAMRefiner python=3.8 -y
conda activate SAMRefiner

# install packages
pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 --extra-index-url https://download.pytorch.org/whl/cu113
pip install opencv-python tqdm matplotlib scipy
pip install FastGeodis --no-build-isolation --no-cache-dir

cd SAMRefiner_repo/segment-anything; pip install -e .; cd ../../
```

* Download model checkpoint

```bash
wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -O SAMRefiner_repo/checkpoints
```

In [ ]:
import sys
import numpy as np
from PIL import Image
sys.path.append("SAMRefiner_repo")

from segment_anything import sam_model_registry
from sam_refiner import sam_refiner

/home/pyml/anaconda3/envs/SAMRefiner/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_type = "vit_h"
sam_checkpoint = "SAMRefiner_repo/checkpoints/sam_vit_h_4b8939.pth"
device = "cuda"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.eval().to(device=device);